In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=15d1f44be481edd2e9f29c1ef389b1442f00a9782befed1a70d012ab0c29801f
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setAppName("appName").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [ ]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df


DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [ ]:
#Question NO.1
my_list = list(range(1, 11))
print("Original list:", my_list)

squared_list = list(map(lambda x: x**2, my_list))
print("Squared list:", squared_list)


Original list: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Squared list: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [ ]:
#Question No.2
import random
random_list = []
for i in range(20):
    random_list.append(random.randint(1, 100))

print("Random list:", random_list)

divisible_by_5 = list(filter(lambda x:(x%5 == 0) , random_list))
print("Updated : " , divisible_by_5)

Random list: [80, 6, 72, 6, 21, 15, 77, 23, 36, 34, 76, 6, 63, 53, 80, 64, 31, 8, 49, 26]
Updated :  [80, 15, 80]


In [ ]:
#Question No.3
from pyspark.sql.functions import split,collect_list,col,regexp_extract,length
#Part (a)
file=spark.read.text('hi.txt')
print("File \n")
file.show()

#Part(b)
print("Comments By Each User \n")

Comments = file.withColumn("UserName", split(col("value"), ", ")[0]).withColumn("Comment", split(col("value"), ", ")[1])
Final_Comments = Comments.groupBy("UserName").agg(collect_list("Comment").alias("comments"))
Final_Comments.show()

#Part(C)
print("Long Comments > 20 \n")

Long_Comments = Comments.filter(length(Comments.Comment) > 20).groupBy("UserName").count()
Long_Comments.show()

#Part(D)
print("Name starting with English Alphabet \n")

Name_counts = Final_Comments.select(regexp_extract('UserName', '^[A-Za-z]', 0).alias('first_letter')).groupBy('first_letter').count().orderBy('first_letter')
Name_counts.show()

#Part(E)
Num_Comments = Comments.groupBy("UserName").count()
Max_Comment = Num_Comments.orderBy(col("count").desc()).first()["UserName"]

print("The User With Maximum Number Of Comments Is:", Max_Comment)


File 

+--------------------+
|               value|
+--------------------+
|   UserName, Comment|
|                    |
|Aliya153, Your we...|
|Sara2, You need t...|
|     Ali45, Good !!!|
|Ali45, I will def...|
+--------------------+

Comments By Each User 

+--------+--------------------+
|UserName|            comments|
+--------+--------------------+
|UserName|           [Comment]|
|   Ali45|[Good !!!, I will...|
|Aliya153|[Your website is ...|
|   Sara2|[You need to work...|
|        |                  []|
+--------+--------------------+

Long Comments > 20 

+--------+-----+
|UserName|count|
+--------+-----+
|   Ali45|    1|
|Aliya153|    1|
|   Sara2|    1|
+--------+-----+

Name starting with English Alphabet 

+------------+-----+
|first_letter|count|
+------------+-----+
|            |    1|
|           A|    2|
|           S|    1|
|           U|    1|
+------------+-----+

The User With Maximum Number Of Comments Is: Ali45


In [ ]:
#Question No.4
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

with open('hi.txt', 'r') as f:
    Comments = f.readlines()

for comment in Comments:
    comment = comment.strip().split(',')[-1].strip() 
    words = comment.split()  
    filtered_words = [word for word in words if word.lower() not in stop_words] 
    filtered_comment = ' '.join(filtered_words)
    print(filtered_comment)


Comment

website superb
need work website design
Good !!!
definitely visit


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
